In [7]:
!pip install docx2txt
import docx2txt
 
def extract_text_from_doc(doc_path):
    temp = docx2txt.process(doc_path)
    text = [line.replace('\t', ' ') for line in temp.split('\n') if line]
    return ' '.join(text)

# Training

In [12]:
import spacy
import random
import json

training_data = []
with open("traindata.json","rb") as f:
  lines = f.readlines()
  for line in lines:
    train_data = json.loads(line)   
    text = train_data['content']
    entities = []
    for annotation in train_data['annotation']:
        point = annotation['points'][0]
        labels = annotation['label']
        if not isinstance(labels, list):
            labels = [labels]
        flag = True
        for label in labels:
            for x in entities:
              if point['start'] <= x[1]:
                flag = False
            if flag:
              entities.append((point['start'], point['end'] + 1 ,label))
    training_data.append((text, {"entities" : entities}))

TRAINING_DATA = training_data

FileNotFoundError: [Errno 2] No such file or directory: 'traindata.json'

In [3]:
import re
invalid_span_tokens = re.compile(r'\s')
cleaned_data = []
for text, annotations in training_data:
    entities = annotations['entities']
    valid_entities = []
    for start, end, label in entities:
        valid_start = start
        valid_end = end
        while valid_start < len(text) and invalid_span_tokens.match(
                text[valid_start]):
            valid_start += 1
        while valid_end > 1 and invalid_span_tokens.match(
                text[valid_end - 1]):
            valid_end -= 1
        valid_entities.append([valid_start, valid_end, label])
    cleaned_data.append([text, {'entities': valid_entities}])

In [4]:
nlp = spacy.blank("en")
ner = nlp.create_pipe("ner")
nlp.add_pipe(ner)

for _, annotations in TRAINING_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

In [5]:
nlp.begin_training()
for itn in range(20):
    random.shuffle(cleaned_data)
    losses = {}
    for text, annotations in cleaned_data:
        nlp.update([text], [annotations], losses=losses)
    print(losses)

C:\Users\vidushi jha\anaconda3\lib\site-packages\spacy\language.py:639: UserWarning: [W033] Training a new parser or NER using a model with no lexeme normalization table. This may degrade the performance of the model to some degree. If this is intentional or the language you're using doesn't have a normalization table, please ignore this warning. If this is surprising, make sure you have the spacy-lookups-data package installed. The languages with lexeme normalization tables are currently: da, de, el, en, id, lb, pt, ru, sr, ta, th.
  **kwargs
C:\Users\vidushi jha\anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Soumya Balan
IT SUPPORT

Sulthan Bathery, Kerala, ..." with entities "[[4167, 4176, 'Companies worked at']]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\vidus

C:\Users\vidushi jha\anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Vineeth Vijayan
"Store Executive" - Orange City Ho..." with entities "[[6994, 7350, 'Skills']]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\vidushi jha\anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Paul Rajiv
Secunderabad, Andhra Pradesh - Email me..." with entities "[[4729, 4733, 'Graduation Year']]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\vidushi jha\anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Fenil

C:\Users\vidushi jha\anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Shraddha Achar
Mathura, Uttar Pradesh - Email me o..." with entities "[[975, 1020, 'Skills']]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\vidushi jha\anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Divesh Singh
Bengaluru, Karnataka - Email me on In..." with entities "[[948, 1180, 'Skills']]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\vidushi jha\anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Keshav Dhawale
3

{'ner': 9493.369426692108}
{'ner': 6065.3151344938}
{'ner': 3513.741125900381}
{'ner': 3307.208260389254}
{'ner': 2691.8973101926176}
{'ner': 2795.1890996879897}
{'ner': 3538.053716490837}
{'ner': 1455.7594346445587}
{'ner': 1935.2298039431994}
{'ner': 2184.001531122229}
{'ner': 1704.190471654981}
{'ner': 1488.2335152511562}
{'ner': 1342.2059711593731}
{'ner': 1947.163308975708}
{'ner': 1357.8855074084543}
{'ner': 854.2722095737478}
{'ner': 2291.330727753465}
{'ner': 1029.8804961173994}
{'ner': 865.8739269471082}
{'ner': 1573.6939014621305}


# Extracting 

In [45]:
!pip install pdfminer.six    
import io
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, caching=True, check_extractable=True):
            resource_manager = PDFResourceManager()
            fake_file_handle = io.StringIO()
            converter = TextConverter(
                                resource_manager, 
                                fake_file_handle, 
                                codec='utf-8', 
                                laparams=LAParams()
                        )
            page_interpreter = PDFPageInterpreter(
                                resource_manager, 
                                converter
                            )
            page_interpreter.process_page(page)
            text = fake_file_handle.getvalue()
            yield text
            converter.close()
            fake_file_handle.close()
file_path ="Vidushi Jha.pdf"
text = ""
for page in extract_text_from_pdf(file_path):
    text += ' ' + page
print(text)

ERROR: Invalid requirement: '#'


 VIDUSHI JHA 

House no. 79, Tilak Enclave, Mohan Garden, New Delhi, 
110059 | 8505991052 | vidushijha2397@gmail.com 

OBJECTIVE 

A fresher looking for a challenging role in a reputable organization to utilize 
my technical, database and management skills for the growth of the 
organization as well as to enhance my knowledge about new and emerging 
trends in the IT sector. 

SKILLS & ABILITIES 

•  TECHNICAL SKILLS 

Python, C/C++, Scala, Java, SQL, Data Warehousing, ETL Operation, Spark, 
Hive, Big Data, Tableau, Hadoop, AWZ, Microsoft Azure 

•  OPERATING SYTEMS 

Windows, Linux (Ubuntu, Redhat and kali) 

• 

 NON-TECHNICAL SKILLS 

Leadership, Management, Team Spirit, Highly Motivated, Analytical 
Thinking. 

TRAININGS AND INTERNSHIPS 

June, 2018 – 
July, 2019   

Dec, 2018 – Jan, 
2019  

May, 2019-
June, 2019 

 

 

AWS Solution Architect, Intaglio Solutions (Delhi) 

· In this particular training I have learnt and practiced on various AWS tools, 
services and also build vario

In [46]:
doc_to_test = nlp(text)
d={}
for ent in doc_to_test.ents:
    d[ent.label_]=[]
for ent in doc_to_test.ents:
    d[ent.label_].append(ent.text)

In [47]:
print(d)

{'ORG': ['VIDUSHI JHA', 'House', 'Tilak Enclave', 'C/C++', 'Scala, Java', 'SQL', 'Data Warehousing', 'Hive', 'Big Data', 'Tableau, Hadoop', 'AWZ', 'Microsoft', 'Ubuntu', 'Analytical \n', 'TRAININGS', 'INTERNSHIPS', 'Data Analytics', 'Big Data', 'Celebal Technologies', 'Visual Studios', 'SQL', 'Microsoft Azure', 'Global Institute of Technology', 'CGPA', 'Technical', 'Technical Skills'], 'DATE': ['no.', '110059', '8505991052', 'June', '2018', 'July', '2019', '2018', '2019', 'June', '2019', 'June', '2018', 'July', '2019', '2016-2010', '2015-2016', '2013-2014'], 'CARDINAL': ['79', '7.5', '#', '9.8'], 'PERSON': ['Mohan Garden', 'Jan', 'Intaglio Solutions', 'AWS', 'ADHOC Networks', 'AWS', 'Intaglio Solutions', 'AWS', 'Jaipur Rajasthan', 'Gagan Bharti Sr', 'Sec', 'Gagan Bharti Sr.', 'Sec'], 'GPE': ['New Delhi', 'Delhi', 'Jaipur', 'Rajasthan', 'Jaipur', 'Rajasthan', 'Delhi', 'New Delhi', 'New Delhi'], 'PERCENT': ['61%', '81%'], 'ORDINAL': ['10th'], 'PRODUCT': ['Page 2']}


#////////////////////////////////////

## Name 


In [51]:
from spacy.matcher import Matcher
nlp = spacy.load('en_core_web_sm')
matcher = Matcher(nlp.vocab)
def extract_name(resume_text):
    nlp_text = nlp(resume_text)
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    matcher.add('NAME', None, *pattern)
    matches = matcher(nlp_text)
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text

## Phone Number 

In [42]:
import re

def extract_mobile_number(text):
    phone = re.findall(re.compile(r'(?:(?:\+?([1-9]|[0-9][0-9]|[0-9][0-9][0-9])\s*(?:[.-]\s*)?)?(?:\(\s*([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|([0-9][1-9]|[0-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?([2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?([0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?'), text)
    
    if phone:
        number = ''.join(phone[0])
        if len(number) > 10:
            return '+' + number
        else:
            return number

## Email 

In [52]:
def extract_email(email):
    email = re.findall("([^@|\s]+@[^@]+\.[^@|\s]+)", email)
    if email:
        try:
            return email[0].split()[0].strip(';')
        except IndexError:
            return None

## Educaction

In [6]:
import spacy
import nltk
from nltk.corpus import stopwords
stopwords = nltk.download('stopwords')
nlp = spacy.load('en_core_web_sm')
STOPWORDS = set(stopwords.words('english'))
EDUCATION = [  
            'BTECH', 'B.TECH', 'M.TECH', 'MTECH', 
            'SSC', 'HSC', 'CBSE', 'ICSE', '10th', '12th'
        ]

def extract_education(resume_text):
    nlp_text = nlp(resume_text)
    nlp_text = [sent.string.strip() for sent in nlp_text.sents]
    edu = {}
    for index, text in enumerate(nlp_text):
        for tex in text.split():
            tex = re.sub(r'[?|$|.|!|,]', r'', tex)
            if tex.upper() in EDUCATION and tex not in STOPWORDS:
                edu[tex] = text + nlp_text[index + 1]

    education = []
    education = education.astype('string')
    for key in edu.keys():
        year = re.search(re.compile(r'(((20|19)(\d{2})))'), edu[key])
        if year:
            education.append((key, ''.join(year[0])))
        else:
            education.append(key)
    return education

[nltk_data] Downloading package stopwords to C:\Users\vidushi
[nltk_data]     jha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


AttributeError: 'bool' object has no attribute 'words'

## Skill

In [54]:
import pandas as pd
nlp = spacy.load('en_core_web_sm')
noun_chunks = nlp.noun_chunks()
 
def extract_skills(resume_text):
    nlp_text = nlp(resume_text)
    tokens = [token.text for token in nlp_text if not token.is_stop]
    data = pd.read_csv("skills.csv") 
    skills = list(data.columns.values)
    skillset = []
    for token in tokens:
        if token.lower() in skills:
            skillset.append(token)
    for token in noun_chunks:
        token = token.text.lower().strip()
        if token in skills:
            skillset.append(token)
    
    return [i.capitalize() for i in set([i.lower() for i in skillset])]

AttributeError: 'English' object has no attribute 'noun_chunks'

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from spacy.gold import GoldParse
from spacy.scorer import Scorer
from sklearn.metrics import accuracy_score
examples = convert_dataturks_to_spacy("testdata.json")
    tp=0
    tr=0
    tf=0

    ta=0
    c=0        
    for text,annot in examples:

        f=open("resume"+str(c)+".txt","w")
        doc_to_test=nlp(text)
        d={}
        for ent in doc_to_test.ents:
            d[ent.label_]=[]
        for ent in doc_to_test.ents:
            d[ent.label_].append(ent.text)

        for i in set(d.keys()):

            f.write("\n\n")
            f.write(i +":"+"\n")
            for j in set(d[i]):
                f.write(j.replace('\n','')+"\n")
        d={}
        for ent in doc_to_test.ents:
            d[ent.label_]=[0,0,0,0,0,0]
        for ent in doc_to_test.ents:
            doc_gold_text= nlp.make_doc(text)
            gold = GoldParse(doc_gold_text, entities=annot.get("entities"))
            y_true = [ent.label_ if ent.label_ in x else 'Not '+ent.label_ for x in gold.ner]
            y_pred = [x.ent_type_ if x.ent_type_ ==ent.label_ else 'Not '+ent.label_ for x in doc_to_test]  
            if(d[ent.label_][0]==0):
                #f.write("For Entity "+ent.label_+"\n")   
                #f.write(classification_report(y_true, y_pred)+"\n")
                (p,r,f,s)= precision_recall_fscore_support(y_true,y_pred,average='weighted')
                a=accuracy_score(y_true,y_pred)
                d[ent.label_][0]=1
                d[ent.label_][1]+=p
                d[ent.label_][2]+=r
                d[ent.label_][3]+=f
                d[ent.label_][4]+=a
                d[ent.label_][5]+=1
        c+=1
    for i in d:
        print("\n For Entity "+i+"\n")
        print("Accuracy : "+str((d[i][4]/d[i][5])*100)+"%")
        print("Precision : "+str(d[i][1]/d[i][5]))
        print("Recall : "+str(d[i][2]/d[i][5]))
        print("F-score : "+str(d[i][3]/d[i][5]))
train_spacy()